In [1]:
### load package
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import gurobipy as gp
import time
model_sanitized_path = 'C:/Users/52427/Documents/Cases/Case11_Baixiang/Baixiang_supply_chain_optimization/model_sanitized/'
from datetime import date

today = date.today()
today = str(today).replace('-','')



### Load input tables

In [2]:
file_name = model_sanitized_path + 'model_input_20220622_EN.xlsx'
xls = pd.ExcelFile(file_name)

periods_df = pd.read_excel(xls, 'Input_PeriodList')

customers_df = pd.read_excel(xls, 'Input_CustomerList')
### clean customers min perc demand satisfied: cap at 100% & set null to 100% by default 
customers_df.loc[customers_df['MinimumPercDemandSatisfied']>1, 'MinimumPercDemandSatisfied'] = 1
customers_df.loc[customers_df['MinimumPercDemandSatisfied'].isnull(), 'MinimumPercDemandSatisfied'] = 1

sites_df = pd.read_excel(xls, 'Input_SiteList')

productionlines_df = pd.read_excel(xls, 'Input_ProductionLineList')

products_df = pd.read_excel(xls, 'Input_ProductList')

demand_df = pd.read_excel(xls, 'Input_CustomerDemand')

trans_policies_df = pd.read_excel(xls, 'Input_TransportationPolicy')

prod_policies_df = pd.read_excel(xls, 'Input_ProductionPolicy')

inv_df = pd.read_excel(xls, 'Input_InventoryPolicy')

### load baseline data tables
hist_outbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistCustomerFlows')

hist_inbound_trans_df = pd.read_excel(xls, sheet_name='Input_HistIntersiteFlows')

hist_prod_df = pd.read_excel(xls, sheet_name='Input_HistProduction')

xls.close()

In [3]:
### print data info
print('# customers:', customers_df.shape[0])
print('# sites:', sites_df.shape[0])
print('# productionlines:', productionlines_df.shape[0])
print('# products:', products_df.shape[0])
print('# customer-product combinations in demand table:', demand_df.shape[0])
print('# customers in demand table:', len(demand_df['CustomerName'].unique()))
print('# products in demand table:', len(demand_df['ProductName'].unique()))
print('# all lanes:', trans_policies_df[['Origin','Destination']].drop_duplicates().shape[0])
print('# origins in transportation table:', trans_policies_df[['Origin']].drop_duplicates().shape[0])
print('# destinations in transportation table:', trans_policies_df[['Destination']].drop_duplicates().shape[0])
print('# line-product combinations:',prod_policies_df.shape[0])
print('# lines:',prod_policies_df[['ProductionLine']].drop_duplicates().shape[0])

### print baseline data info
print('# dcs to customers:', hist_outbound_trans_df.shape[0])
print('# factories to dcs:', hist_inbound_trans_df.shape[0])
print('# production combs:', hist_prod_df.shape[0])

# customers: 293
# sites: 18
# productionlines: 59
# products: 61
# customer-product combinations in demand table: 34115
# customers in demand table: 293
# products in demand table: 61
# all lanes: 2718
# origins in transportation table: 18
# destinations in transportation table: 302
# line-product combinations: 187
# lines: 59
# dcs to customers: 34932
# factories to dcs: 506
# production combs: 1642


### Preprocess

In [4]:
### create lists of basic model elements
customers = [x for x in customers_df['CustomerName']]

dcs = [x for x in sites_df['SiteName'][sites_df['SiteName'].str.contains('DC')]]

factories = [x for x in sites_df['SiteName'][sites_df['SiteName'].str.contains('FAC')]]

productionlines = [(x['FactoryName'],x['ProductionLine']) for _, x in productionlines_df.iterrows()]

products = [x for x in products_df['ProductName']]

periods = [x for x in periods_df['PeriodName']]

n_periods = len(periods)

In [5]:
### product type
prod_type = products_df.groupby(['ProductName'])['ProductType'].max().to_dict()

### customer demand
cust_demand = demand_df.groupby(['CustomerName', 'ProductName', 'Period'])['CustomerDemand'].sum().to_dict()
overall_cust_demand = demand_df.groupby(['CustomerName'])['CustomerDemand'].sum().to_dict()

### customer demand min percsatisfied
min_perc_dmd_satisfied = customers_df.groupby(['CustomerName'])['MinimumPercDemandSatisfied'].max().to_dict()

### production capacity
line_product_rate = prod_policies_df.groupby(['FactoryName', 'ProductionLine', 'ProductName'])['MachineHoursPerUnit'].sum().to_dict()

line_capacity_cap = productionlines_df.groupby(['FactoryName', 'ProductionLine'])['MachineHourCapacityPerPeriod'].sum().to_dict()

line_product_dict = prod_policies_df.groupby(['FactoryName','ProductionLine'])['ProductName'].apply(list).to_dict()

### factory-attached warehouse storage capacity
site_storage_cap = sites_df[sites_df['StorageCapacity'].notnull()].groupby(['SiteName'])['StorageCapacity'].max().to_dict()

### factory-attached warehouse handling capacity
site_handling_cap = sites_df[sites_df['HandlingCapacityPerPeriod'].notnull()].groupby(['SiteName'])['HandlingCapacityPerPeriod'].max().to_dict()

### factory variable production cost
var_prod_cost = prod_policies_df.groupby(['FactoryName', 'ProductionLine', 'ProductName'])['VariableProductionCost'].sum().to_dict()

### dc variable handling cost 
var_handling_cost = sites_df.groupby(['SiteName'])['VariableHandlingCost'].sum().to_dict()

### factory fixed cost
site_fixed_cost = sites_df.groupby(['SiteName'])['FixedOperatingCostPerHorizon'].sum().to_dict()

### product line fixed cost
productline_fixed_cost = productionlines_df.groupby(['FactoryName','ProductionLine'])['FixedOperatingCostPerHorizon'].sum().to_dict()

### transportation cost
transp_cost = trans_policies_df.groupby(['Origin', 'Destination', 'ProductType'])['VariableTransportationCost'].max().to_dict()

### transportation distance
transp_dist = trans_policies_df.groupby(['Origin', 'Destination', 'ProductType'])['Distance'].max().to_dict()

### unit product cubic meters
product_cubic = products_df.groupby(['ProductName'])['Volume'].max().to_dict()

### inventory turns
inv_turns = inv_df.groupby(['SiteName','ProductName'])['InventoryTurns'].max().to_dict()

### initial inventory
initial_inv = inv_df.groupby(['SiteName','ProductName'])['InitialInventory'].max().to_dict()

In [6]:
### historical factory-to-customer flows
hist_fc_flows = hist_outbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical inter-factory flows
hist_ff_flows = hist_inbound_trans_df.groupby(
    ['Origin', 'Destination', 'ProductName', 'Period'])['HistoricalShipment'].sum().to_dict()

### historical production
hist_prod_flows = hist_prod_df.groupby(
    ['FactoryName', 'ProductionLine', 'ProductName', 'Period'])['HistoricalProduction'].sum().to_dict()

In [7]:
### generate index sets
ss_lanes = gp.tuplelist([(o, d, p, m) for o in factories for d in dcs for p in products for m in periods]) 

sc_lanes = gp.tuplelist([(o, d, p, m) for o in dcs for (d, p, m) in cust_demand]) 

# prod_policies_period_df = pd.merge(prod_policies_df[['FactoryName', 'ProductionLine', 'ProductName']].assign(temp=1), 
#                                   pd.DataFrame(periods, columns =['Period']).assign(temp=1), on='temp').drop('temp', axis=1)
# prod_policies = gp.tuplelist(zip(prod_policies_period_df['FactoryName'], prod_policies_period_df['ProductionLine'], 
#                                  prod_policies_period_df['ProductName'], prod_policies_period_df['Period']))
prod_policies = gp.tuplelist([(f, l, p, m) for (f, l, p) in line_product_rate for m in periods])


dc_inv_basis = gp.tuplelist([(dc, p, m) for dc in dcs for p in products for m in list(periods) + [max(periods)+1]])

f_inv_basis = gp.tuplelist([(f, p, m) for f in factories for p in products for m in list(periods) + [max(periods)+1]])

### per unit penalty cost of using dummy lanes
dmd_pen = np.ceil(max(var_prod_cost.values())*10 + max(var_handling_cost.values())*10 + max(transp_cost.values())*10)

### Scenario setting

In [8]:
# scenarios matrix
scenario_list = ['Baseline2021','Unconstrainted','CloseOneFactory','Unconstrainted+RemoveStorageCons','Unconstrainted+RemoveHandlingCons']
constr_setting = {'dc_handling_cap':[False, True, True, True, False], 
                  'dc_storage_cap':[False, True, True, False, True],
                  'factory_handling_cap':[False, True, True, True, False], 
                  'factory_storage_cap':[False, True, True, False, True],
                  'perc_demand_satisfied':[False, False, False, False, False],
                  'hist_fc':[True, False, False, False, False], 
                  'hist_ff':[True, False, False, False, False], 
                  'hist_prod':[True, False, False, False, False], 
                  'hist_no_prod':[True, False, False, False, False], 
                  'fix_factories_open':[True, True, True, True, True],
                  'fix_dcs_open':[True, True, True, True, True],
                  'num_factories_open':[9, 9, 8, 9, 9], 
                  'num_dcs_open':[9, 9, 8, 9, 9], 
                  'dc_initial_inv': [True, True, True, True, True],
                  'factory_initial_inv': [True, True, True, True, True]}
scenario_matrix = pd.DataFrame(constr_setting, index=scenario_list)
scenario_matrix

,dc_handling_cap,dc_storage_cap,factory_handling_cap,factory_storage_cap,perc_demand_satisfied,hist_fc,hist_ff,hist_prod,hist_no_prod,fix_factories_open,fix_dcs_open,num_factories_open,num_dcs_open,dc_initial_inv,factory_initial_inv
Baseline2021,False,False,False,False,False,True,True,True,True,True,True,9,9,True,True
Unconstrainted,True,True,True,True,False,False,False,False,False,True,True,9,9,True,True
CloseOneFactory,True,True,True,True,False,False,False,False,False,True,True,8,8,True,True
Unconstrainted+RemoveStorageCons,True,False,True,False,False,False,False,False,False,True,True,9,9,True,True
Unconstrainted+RemoveHandlingCons,False,True,False,True,False,False,False,False,False,True,True,9,9,True,True


### Network Optimization Model

In [9]:
#scenario = 'Baseline2021'
#scenario = 'Unconstrainted'
#scenario = 'CloseOneFactory'
#scenario = 'Unconstrainted+RemoveStorageCons'
scenario = 'Unconstrainted+RemoveHandlingCons'

In [10]:
time_start = time.time()

### create model
m = gp.Model('BXNetworkOpt')

### decision variables
ss_flow = m.addVars(ss_lanes, vtype=gp.GRB.CONTINUOUS, name='ff_flow') # factory to dc flows
sc_flow = m.addVars(sc_lanes, vtype=gp.GRB.CONTINUOUS, name='fc_flow') # dc to customer flows
prod_flow = m.addVars(prod_policies, vtype=gp.GRB.CONTINUOUS, name='prod_flow') # production volume
f_inv_level = m.addVars(f_inv_basis, vtype=gp.GRB.CONTINUOUS, name='f_inv_level') # factory beginning inventory level of each period
dc_inv_level = m.addVars(dc_inv_basis, vtype=gp.GRB.CONTINUOUS, name='dc_inv_level') # dc beginning inventory level of each period
f_open = m.addVars(factories, vtype=gp.GRB.BINARY, name='f_open') # factory open or not
dc_open = m.addVars(dcs, vtype=gp.GRB.BINARY, name='dc_open') # dc open or not
productline_open = m.addVars(productionlines, vtype=gp.GRB.BINARY, name='dc_open') # product line open or not
demand_slack = m.addVars(cust_demand, vtype=gp.GRB.CONTINUOUS, name='demand_slack') # slack variable for unmatched demand

m.update()

### objective function
tot_var_prod_cost = gp.quicksum(prod_flow[(f, l, p, m)]*var_prod_cost[(f, l, p)] for (f, l, p, m) in prod_policies)
tot_dc_var_handling_cost = gp.quicksum(sc_flow[(o, d, p, m)]*var_handling_cost[(o)] for (o, d, p, m) in sc_lanes)
tot_f_var_handling_cost = gp.quicksum(ss_flow[(o, d, p, m)]*var_handling_cost[(o)] for (o, d, p, m) in ss_lanes) 
tot_site_fixed_cost = gp.quicksum(f_open[f]*site_fixed_cost[f] for f in factories) + \
                        gp.quicksum(dc_open[dc]*site_fixed_cost[dc] for dc in dcs) + \
                        gp.quicksum(productline_open[(f, l)]*productline_fixed_cost[(f, l)] for (f, l) in productionlines)
tot_ss_transp_cost = gp.quicksum(ss_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d, prod_type[p])] for (o, d, p, m) in ss_lanes)
tot_sc_transp_cost = gp.quicksum(sc_flow[(o, d, p, m)]*product_cubic[p]*transp_cost[(o, d, prod_type[p])] for (o, d, p, m) in sc_lanes)
tot_dmd_pen_cost = demand_slack.sum('*', '*', '*')*dmd_pen
tot_cost = tot_var_prod_cost + tot_dc_var_handling_cost + tot_f_var_handling_cost + tot_site_fixed_cost + tot_ss_transp_cost + tot_sc_transp_cost + tot_dmd_pen_cost

m.setObjective(tot_cost, gp.GRB.MINIMIZE)

### demand satisfaction
### soft demand satisfication cons
m.addConstrs(
    (sc_flow.sum('*', c, p, m) == cust_demand[(c, p, m)] + demand_slack[(c, p, m)] for (c, p, m) in cust_demand.keys()), 'customer_demand'
)

### at least x% demand must be satisfied for each customer
if scenario_matrix.loc[scenario, 'perc_demand_satisfied']:
    m.addConstr(
        (sc_flow.sum('*', c, '*', '*') >= 0.8 * sum(overall_cust_demand[c]) for c in customers), 'perc_demand_satisfied'
    )

### flow balance
### factory flow balance
m.addConstrs(
    (f_inv_level[(f, p, m)] + prod_flow.sum(f, '*', p, m) == ss_flow.sum(f, '*', p, m) + f_inv_level[(f, p, m+1)] for f in factories for p in products for m in periods), 'flow_balance'
)
### dc flow balance
m.addConstrs(
    (dc_inv_level[(dc, p, m)] + ss_flow.sum('*', dc, p, m) == sc_flow.sum(dc, '*', p, m) + dc_inv_level[(dc, p, m+1)]
     for dc in dcs for p in products for m in periods), 'flow_balance'
)

### factory production capacity by line
m.addConstrs(
    (gp.quicksum([prod_flow[(f, l, p, m)] * line_product_rate[(f, l, p)] for p in line_product_dict[(f, l)]])<= line_capacity_cap[(f, l)]*productline_open[(f, l)] for (f, l) in productionlines for m in periods), 'line_capacity_cap'
)

### product line open
m.addConstrs(
    (productline_open[(f, l)] <= f_open[f] for (f, l) in productionlines), 'productline_open'
)

### dc handling capacity
if scenario_matrix.loc[scenario, 'dc_handling_cap']:
    m.addConstrs(
        (sc_flow.sum(dc, '*', '*', m) 
         <= site_handling_cap[dc]*dc_open[dc] for dc in set(dcs).intersection(site_handling_cap.keys()) for m in periods), 'dc_handling_cap'
    )

### dc storage capacity
if scenario_matrix.loc[scenario, 'dc_storage_cap']:
    m.addConstrs(
        (dc_inv_level.sum(dc, '*', m) 
         <= site_storage_cap[dc]*dc_open[dc] for dc in set(dcs).intersection(site_storage_cap.keys()) for m in periods), 'dc_storage_cap'
    )

### factory handling capacity
if scenario_matrix.loc[scenario, 'factory_handling_cap']:
    m.addConstrs(
        (ss_flow.sum(f, '*', '*', m) 
         <= site_handling_cap[f]*f_open[f] for f in set(factories).intersection(site_handling_cap.keys()) for m in periods), 'factory_handling_cap'
    )

### factory storage capacity
if scenario_matrix.loc[scenario, 'factory_storage_cap']:
    m.addConstrs(
        (f_inv_level.sum(f, '*', m) 
         <= site_storage_cap[f]*f_open[f] for f in set(factories).intersection(site_storage_cap.keys()) for m in periods), 'factory_storage_cap'
    )



### number of factories open
if scenario_matrix.loc[scenario, 'fix_factories_open']:
    m.addConstr(
        (f_open.sum('*') == scenario_matrix.loc[scenario, 'num_factories_open']), 'num_factories_open'
    )

### number of dcs open
if scenario_matrix.loc[scenario, 'fix_dcs_open']:
    m.addConstr(
        (dc_open.sum('*') == scenario_matrix.loc[scenario, 'num_dcs_open']), 'num_dcs_open'
    )

### dc inital inventory
if scenario_matrix.loc[scenario, 'dc_initial_inv']:
    m.addConstrs(
        (dc_inv_level[(dc, p, 1)] == initial_inv[(dc, p)] for dc in dcs for p in products), 'dc_intial_inv'
    )

### factory inital inventory
if scenario_matrix.loc[scenario, 'factory_initial_inv']:
    m.addConstrs(
        (f_inv_level[(f, p, 1)] == initial_inv[(f, p)] for f in factories for p in products), 'f_intial_inv'
    )

##########################historical cons
### historical factory-to-customer flows
if scenario_matrix.loc[scenario, 'hist_fc']:
    m.addConstrs(
        (sc_flow[(o, d, p, m)] == hist_fc_flows[(o, d, p, m)] for (o, d, p, m) in hist_fc_flows.keys()), 'hist_fc'
    )

### historical inter-factory flows
if scenario_matrix.loc[scenario, 'hist_ff']:
    m.addConstrs(
        (ss_flow[(o, d, p, m)] == hist_ff_flows[(o, d, p, m)] for (o, d, p, m) in hist_ff_flows.keys()), 'hist_ff'
    )

### historical production
if scenario_matrix.loc[scenario, 'hist_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == hist_prod_flows[(f, l, p, m)] for (f, l, p, m) in hist_prod_flows.keys()), 'hist_prod'
    )

### historical no production
if scenario_matrix.loc[scenario, 'hist_no_prod']:
    m.addConstrs(
        (prod_flow[(f, l, p, m)] == 0 for (f, l, p, m) in [x for x in prod_policies if x not in hist_prod_flows.keys()]), 
        'hist_no_prod'
    )



time_build_end = time.time()

### solve the model
m.optimize()
time_solve_end = time.time()

print()
print('optimal' if m.status==gp.GRB.OPTIMAL else 'infeasible')
print('model build time:', round(time_build_end - time_start), 's')
print('model solve time:', round(time_solve_end - time_build_end), 's')
tot_oprt_cost = tot_var_prod_cost.getValue() + tot_dc_var_handling_cost.getValue() + tot_f_var_handling_cost.getValue() \
                + tot_site_fixed_cost.getValue() + tot_sc_transp_cost.getValue() + tot_ss_transp_cost.getValue()
print('total operating cost:', tot_oprt_cost)

### 11.5 mins for unconstraint+removeHandlingCons scenario: 33w continous 44 intergers

Set parameter CloudAccessID
Set parameter CloudSecretKey
Set parameter CloudPool to value "800758-SCORteamCOMMON"
Set parameter CSAppName to value "SCOR Team Common License"
Set parameter LicenseID
Waiting for cloud server to start (pool 800758-SCORteamCOMMON)...
Starting...
Starting...
Starting...
Compute Server job ID: 74846e19-1ee3-4b5e-a456-90c04f878e89
Capacity available on '800758-SCORteamCOMMON' cloud pool - connecting...
Established HTTPS encrypted connection
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Gurobi Compute Server Worker version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads
Optimize a model with 49374 rows, 417037 columns and 812835 nonzeros
Model fingerprint: 0x52c19ee6
Variable types: 416960 continuous, 77 integer (77 binary)
Coefficient statistics:
  Matrix range     [5e-04, 2e+05]
  Objective range  [4e-01, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 3e+05]
Found he

### Optimization Results

In [ ]:
sc_flow_res = []

for i in sc_lanes:
    if sc_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'ProductType': prod_type[i[2]],
            'Period': i[3],
            'Quantity': sc_flow[i].x
        }
        sc_flow_res.append(var_output)

sc_flow_res_df = pd.DataFrame.from_records(sc_flow_res)

len(sc_flow_res_df) == len(hist_outbound_trans_df)

False

In [ ]:
ss_flow_res = []

for i in ss_lanes:
    if ss_flow[i].x > 0:
        var_output = {
            'Origin': i[0],
            'Destination': i[1],
            'ProductName': i[2],
            'ProductType': prod_type[i[2]],
            'Period': i[3],
            'Quantity': ss_flow[i].x
        }
        ss_flow_res.append(var_output)

ss_flow_res_df = pd.DataFrame.from_records(ss_flow_res)

len(ss_flow_res_df) == len(hist_inbound_trans_df)

False

In [ ]:
prod_flow_res = []

for i in prod_policies:
    if prod_flow[i].x > 0:
        var_output = {
            'FactoryName': i[0],
            'ProductionLine': i[1],
            'ProductName': i[2],
            'ProductType': prod_type[i[2]],
            'Period': i[3],
            'Quantity': prod_flow[i].x
        }
        prod_flow_res.append(var_output)
        
prod_flow_res_df = pd.DataFrame.from_records(prod_flow_res)

len(prod_flow_res_df) == len(hist_prod_df)

False

In [ ]:
dc_inv_level_res = []

for i in dc_inv_basis:
    var_output = {
        'SiteName': i[0],
        'SiteType': 'Distribution Center',
        'ProductName': i[1],
        'ProductType': prod_type[i[1]],
        'Period': i[2],
        'Quantity': dc_inv_level[i].x
    }
    dc_inv_level_res.append(var_output)
    
dc_inv_level_res_df = pd.DataFrame.from_records(dc_inv_level_res)

dc_inv_level_res_df.head()

,SiteName,SiteType,ProductName,ProductType,Period,Quantity
0,DC_1,Distribution Center,PROD_1,AMB,1,0.0
1,DC_1,Distribution Center,PROD_1,AMB,2,0.0
2,DC_1,Distribution Center,PROD_1,AMB,3,0.0
3,DC_1,Distribution Center,PROD_1,AMB,4,0.0
4,DC_1,Distribution Center,PROD_1,AMB,5,0.0


In [20]:
f_inv_level_res = []

for i in f_inv_basis:
    var_output = {
        'SiteName': i[0],
        'SiteType': 'Factory',
        'ProductName': i[1],
        'ProductType': prod_type[i[1]],
        'Period': i[2],
        'Quantity': f_inv_level[i].x
    }
    f_inv_level_res.append(var_output)
        
f_inv_level_res_df = pd.DataFrame.from_records(f_inv_level_res)

f_inv_level_res_df.head()

### concat factory and dc inventory level df
inv_level_res_df = pd.concat([f_inv_level_res_df, dc_inv_level_res_df])
inv_level_res_df = inv_level_res_df[inv_level_res_df['Period']!=1]\
                    .assign(Period = lambda x: x['Period']-1)\
                        .rename({'Quantity' : 'EndOfPeriodInventory'}, axis=1)

In [21]:
demand_slack_res = [] 

for i in cust_demand:
    if demand_slack[i].x > 0:
        var_output = {
            'FactoryName': i[0],
            'ProductName': i[1],
            'ProductType': prod_type[i[1]],
            'Period': i[2],
            'Quantity': demand_slack[i].x
        }
        demand_slack_res.append(var_output)
        
demand_slack_res_df = pd.DataFrame.from_records(demand_slack_res)

if len(demand_slack_res_df)>1:
    print(demand_slack_res_df[['Quantity']].sum())
else:
    print('no unsatisfied demand')

Quantity    7.000750e-13
dtype: float64


In [22]:
f_open_res = []

for i in factories:
    var_output = {
        'SiteName': i,
        'SiteStatus': f_open[i].x
    }
    f_open_res.append(var_output)
        
f_open_res_df = pd.DataFrame.from_records(f_open_res)

f_open_res_df

,SiteName,SiteStatus
0,FAC_1,1.0
1,FAC_2,1.0
2,FAC_3,1.0
3,FAC_4,1.0
4,FAC_5,1.0
5,FAC_6,1.0
6,FAC_7,1.0
7,FAC_8,1.0
8,FAC_9,1.0


In [23]:
dc_open_res = []

for i in dcs:
    var_output = {
        'SiteName': i,
        'SiteStatus': dc_open[i].x
    }
    dc_open_res.append(var_output)
        
dc_open_res_df = pd.DataFrame.from_records(dc_open_res)

dc_open_res_df

,SiteName,SiteStatus
0,DC_1,1.0
1,DC_2,1.0
2,DC_3,1.0
3,DC_4,1.0
4,DC_5,1.0
5,DC_6,1.0
6,DC_7,1.0
7,DC_8,1.0
8,DC_9,1.0


In [24]:
productline_open_res = []

for i in productionlines:
    var_output = {
        'FactoryName': i[0],
        'ProductionLine': i[1],
        'LineStatus': productline_open[i].x
    }
    productline_open_res.append(var_output)
        
productline_open_res_df = pd.DataFrame.from_records(productline_open_res)

productline_open_res_df.head()

,FactoryName,ProductionLine,LineStatus
0,FAC_1,FAC_1_LINE_11,1.0
1,FAC_1,FAC_1_LINE_12,0.0
2,FAC_1,FAC_1_LINE_13,1.0
3,FAC_1,FAC_1_LINE_14,1.0
4,FAC_1,FAC_1_LINE_15,1.0


### Prepare output tables

In [41]:
### add cost of flow and production
sc_flow_res_df_1 = sc_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination','ProductType'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['VariableTransportationCost']})\
            .merge(sites_df[['SiteName','VariableHandlingCost']].rename({'SiteName':'Origin'},axis=1), on = ['Origin'])\
                .assign(**{"HandlingCost": lambda x: x['Quantity']*x['VariableHandlingCost']})


ss_flow_res_df_1 = ss_flow_res_df.merge(trans_policies_df, on = ['Origin','Destination','ProductType'])\
    .merge(products_df[['ProductName','Volume']], on = ['ProductName'])\
        .assign(**{"TransportationCost": lambda x: x['Quantity']*x['Volume']*x['VariableTransportationCost']})\
            .merge(sites_df[['SiteName','VariableHandlingCost']].rename({'SiteName':'Origin'},axis=1), on = ['Origin'])\
                .assign(**{"HandlingCost": lambda x: x['Quantity']*x['VariableHandlingCost']})

prod_flow_res_df_1 = prod_flow_res_df.merge(prod_policies_df, on = ['FactoryName','ProductionLine','ProductName'])\
    .assign(**{"ProductionCost": lambda x: x['Quantity']*x['VariableProductionCost']})\
        .assign(**{'MachineHours': lambda x: x['Quantity'] * x['MachineHoursPerUnit']})

site_open_res_df_1 = sites_df[['SiteName','FixedOperatingCostPerHorizon']].merge(pd.concat([f_open_res_df,dc_open_res_df]), on = 'SiteName', how= 'left')\
    .assign(**{'SiteFixedOperatingCost': lambda x: x['SiteStatus']*x['FixedOperatingCostPerHorizon']})

inv_level_res_df_1 = inv_level_res_df.merge(pd.concat([ss_flow_res_df.groupby(['Origin','ProductName','Period'])['Quantity'].sum().reset_index().rename({'Origin':'SiteName','Quantity':'Throughput'}, axis=1),
                                sc_flow_res_df.groupby(['Origin','ProductName','Period'])['Quantity'].sum().reset_index().rename({'Origin':'SiteName','Quantity':'Throughput'}, axis=1)]),
                                on = ['SiteName', 'ProductName', 'Period'])\
                                    .merge(inv_df[['SiteName', 'ProductName','InventoryTurns']], on = ['SiteName', 'ProductName'])\
                                        .assign(MonthlyInventoryTurns = lambda x: x['InventoryTurns'] / 12,
                                                TurnEstimatedInventory  = lambda x: x['Throughput'] / x['InventoryTurns'])\
                                                    .drop(['InventoryTurns'], axis=1)
                                                    
### period cost summary 
period_cost_summary = sc_flow_res_df_1.groupby(['Period'])[['TransportationCost','HandlingCost','Quantity']].sum().reset_index()\
    .rename({'TransportationCost':'TransportationCostCustomerFlows','HandlingCost':'DCHandlingCost','Quantity':'QuantityCustomerFlows'},axis=1)\
        .merge(ss_flow_res_df_1.groupby(['Period'])[['TransportationCost','HandlingCost','Quantity']].sum().reset_index()\
            .rename({'TransportationCost':'TransportationCostIntersiteFlows','HandlingCost':'FactoryHandlingCost','Quantity':'QuantityIntersiteFlows'},axis=1), on = ['Period'])\
                .assign(**{'TransportationCost': lambda x: (x['TransportationCostCustomerFlows']+x['TransportationCostIntersiteFlows'])})\
                    .merge(prod_flow_res_df_1.groupby(['Period'])[['ProductionCost','Quantity','MachineHours']].sum().reset_index()\
                        .rename({'ProductionCost':'FactoryProductionCost', 'Quantity':'FactoryProductionQuantity'},axis=1), on = ['Period'])\
                            .assign(SiteFixedOperatingCost = site_open_res_df_1[['FixedOperatingCostPerHorizon']].sum()[0]/n_periods, 
                                    NumberOfSitesOpen = site_open_res_df_1[['SiteStatus']].sum()[0])\
                                        .assign(TotalCost = lambda x: x['TransportationCost']+x['DCHandlingCost']+x['FactoryProductionCost']+x['SiteFixedOperatingCost'])\
                                            .merge(demand_df.groupby('Period')['CustomerDemand'].sum().reset_index().rename({'CustomerDemand':'TotalDemand'}, axis=1), on = ['Period'])\
                                                .assign(DemandSatisfiedPerc = lambda x: x['QuantityCustomerFlows'] / x['TotalDemand'])
                                            # .assign(ProductionCapacityUtilization = lambda x: x['MachineHours']/prod_line_policies_df['MachineHourCapacity'].sum(),
                                            #         HandlingCapacityUtilization = lambda x: x['QuantityCustomerFlows']/sites_df['HandlingCapacity'].sum(),
                                            #         ### use inventory turns to calculate periodly storage utilization, is it ok?
                                            #         StorageCapacityUtilization = lambda x: x['QuantityCustomerFlows']/(inv_df['InventoryTurns'].mean()/n_periods*sites_df['StorageCapacity'].sum()))
  
### site cost summary
##### factory cost summary
factory_cost_summary = ss_flow_res_df_1.groupby(['Origin']).apply(lambda x : pd.Series({'TransportationCost': x['TransportationCost'].sum(),
                                                                                        'HandlingCost': x['HandlingCost'].sum(),
                                                                                        'Quantity': x['Quantity'].sum(),
                                                                                        'MaxDistance':x['Distance'].max(),
                                                                                        'WeightedAverageDistance':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
                                                                                        })).reset_index()\
            .rename({'Origin':'SiteName', 'TransportationCost':'OutboundTransportationCost', 'Quantity':'ThoughputLevel'},axis=1)\
                .merge(f_open_res_df.merge(sites_df[['SiteName','FixedOperatingCostPerHorizon']], on = ['SiteName']).assign(FixedOperatingCostPerHorizon = lambda x: x['FixedOperatingCostPerHorizon']*x['SiteStatus']), on = ['SiteName'])\
                    .merge(prod_flow_res_df_1.groupby(['FactoryName'])[['ProductionCost','MachineHours']].sum().reset_index()\
                        .merge(productionlines_df.groupby(['FactoryName']).agg({'MachineHourCapacityPerPeriod':'sum'}).reset_index(), on = ['FactoryName'])\
                            .assign(MachineHourCapacityPerHorizon = lambda x: x['MachineHourCapacityPerPeriod']*n_periods,
                                    ProductionCapacityUtilization = lambda x: x['MachineHours']/x['MachineHourCapacityPerHorizon'])\
                                    .rename({'FactoryName':'SiteName'},axis=1), on = 'SiteName')\
                                        .merge(sites_df[['SiteName','StorageCapacity','HandlingCapacityPerPeriod']], on = ['SiteName'])\
                                            .assign(HandlingCapacityPerHorizon = lambda x: x['HandlingCapacityPerPeriod']*n_periods)\
                                                .merge(inv_level_res_df_1.groupby(['SiteName'])[['TurnEstimatedInventory','EndOfPeriodInventory']].mean().reset_index(), on = ['SiteName'])\
                                                    .assign(HandlingCapacityUtilization = lambda x: x['ThoughputLevel']/x['HandlingCapacityPerHorizon'],
                                                            StorageCapacityUtilization = lambda x: np.where(x['StorageCapacity'] == 0, np.NaN, x['EndOfPeriodInventory']/x['StorageCapacity']))\
                                                                .drop(['MachineHourCapacityPerPeriod','MachineHourCapacityPerHorizon','MachineHours'], axis=1)
                                        
factory_cost_summary.insert(1, 'SiteType', 'Factory')
##### dc cost summary
dc_cost_summary = sc_flow_res_df_1.groupby(['Origin']).apply(lambda x : pd.Series({'TransportationCost': x['TransportationCost'].sum(),
                                                                                    'HandlingCost': x['HandlingCost'].sum(),
                                                                                    'Quantity': x['Quantity'].sum(),
                                                                                    'MaxDistance':x['Distance'].max(),
                                                                                    'WeightedAverageDistance':(x['Quantity']*x['Volume']*x['Distance']).sum() / (x['Quantity']*x['Volume']).sum(),
                                                                                    })).reset_index()\
    .rename({'Origin':'SiteName', 'TransportationCost':'OutboundTransportationCost', 'Quantity':'ThoughputLevel'},axis=1)\
        .merge(dc_open_res_df.merge(sites_df[['SiteName','FixedOperatingCostPerHorizon']], on = ['SiteName']).assign(FixedOperatingCostPerHorizon = lambda x: x['FixedOperatingCostPerHorizon']*x['SiteStatus']), on = ['SiteName'])\
            .merge(sites_df[['SiteName','StorageCapacity','HandlingCapacityPerPeriod']], on = ['SiteName'])\
                .assign(HandlingCapacityPerHorizon = lambda x: x['HandlingCapacityPerPeriod']*n_periods)\
                    .merge(inv_level_res_df_1.groupby(['SiteName'])[['TurnEstimatedInventory','EndOfPeriodInventory']].mean().reset_index(), on = ['SiteName'])\
                        .assign(HandlingCapacityUtilization = lambda x: x['ThoughputLevel']/x['HandlingCapacityPerHorizon'],
                                StorageCapacityUtilization = lambda x: np.where(x['StorageCapacity'] == 0, np.NaN, x['EndOfPeriodInventory']/x['StorageCapacity']))
            
dc_cost_summary.insert(1, 'SiteType', 'Distribution Center')
site_cost_summary = pd.concat([factory_cost_summary, dc_cost_summary]).drop(['HandlingCapacityPerPeriod'], axis=1)

### product line cost summary
productline_cost_summary = prod_flow_res_df_1.groupby(['FactoryName','ProductionLine'])[['ProductionCost','MachineHours']].sum().reset_index()\
    .merge(productline_open_res_df, on = ['FactoryName','ProductionLine'], how = 'outer').fillna(0) \
        .merge(productionlines_df[['FactoryName','ProductionLine','MachineHourCapacityPerPeriod','FixedOperatingCostPerHorizon']].assign(), on = ['FactoryName','ProductionLine'], how = 'outer')\
            .assign(FixedOperatingCost = lambda x: x['FixedOperatingCostPerHorizon'] * x['LineStatus'],
                    MachineHourCapacityPerHorizon = lambda x: x['MachineHourCapacityPerPeriod']*n_periods,
                    ProductionCapacityUtilization = lambda x: x['MachineHours']/x['MachineHourCapacityPerHorizon'])

### Save to Excel

In [44]:
### save model output template
writer = pd.ExcelWriter(model_sanitized_path + 'model_output_'+ today +'_'+scenario+'.xlsx', engine='xlsxwriter')

### output data tables
pd.DataFrame({'Scenario': [scenario],
                'TotalCost':[tot_cost.getValue()-tot_dmd_pen_cost.getValue()],
                'SiteFixedOperatingCost':[tot_site_fixed_cost.getValue()],
                'FactoryProductionCost':[tot_var_prod_cost.getValue()],
                'FactoryHandlingCost':[tot_f_var_handling_cost.getValue()],
                'DCHandlingCost':[tot_dc_var_handling_cost.getValue()],
                'TransportationCost':[tot_sc_transp_cost.getValue()+tot_ss_transp_cost.getValue()],
                'TransportationCostCustomerFlows':[tot_sc_transp_cost.getValue()],
                'TransportationCostIntersiteFlows':[tot_ss_transp_cost.getValue()],
                'DemandSatisfiedPerc': [sc_flow_res_df_1['Quantity'].sum() / demand_df['CustomerDemand'].sum()],
                'TotalDemand':[demand_df['CustomerDemand'].sum()],
                'QuantityCustomerFlows':[sc_flow_res_df_1['Quantity'].sum()],
                'QuantityIntersiteFlows':[ss_flow_res_df_1['Quantity'].sum()],
                'FactoryProductionQuantity':[prod_flow_res_df_1['Quantity'].sum()],
                'NumberOfDCsOpen':[dc_open_res_df['SiteStatus'].sum()],
                'NumberOfFactoriesOpen':[f_open_res_df['SiteStatus'].sum()],
                'MaxDistanceCustomerFlows':[sc_flow_res_df_1['Distance'].max()],
                'WeightedAverageDistanceCustomerFlows':[(sc_flow_res_df_1['Quantity']*sc_flow_res_df_1['Volume']*sc_flow_res_df_1['Distance']).sum() / (sc_flow_res_df_1['Quantity']*sc_flow_res_df_1['Volume']).sum()],
                'WeightedAverageDistanceIntersiteFlows':[(ss_flow_res_df_1['Quantity']*ss_flow_res_df_1['Volume']*ss_flow_res_df_1['Distance']).sum() / (ss_flow_res_df_1['Quantity']*ss_flow_res_df_1['Volume']).sum()]
                })\
        .to_excel(writer, sheet_name='Output_CostSummary', index=False)
period_cost_summary.assign(**{'Scenario': scenario})[['Scenario','Period','TotalCost','SiteFixedOperatingCost','FactoryProductionCost','FactoryHandlingCost',
                        'DCHandlingCost','TransportationCost','TransportationCostCustomerFlows','TransportationCostIntersiteFlows',
                        'DemandSatisfiedPerc','TotalDemand','QuantityCustomerFlows','QuantityIntersiteFlows','FactoryProductionQuantity']] \
                        .to_excel(writer, sheet_name='Output_CostSummaryByPeriod', index=False)
#site_cost_summary.insert(0, 'Scenario', scenario)
site_cost_summary.assign(**{'Scenario': scenario})[['Scenario','SiteName', 'SiteType','SiteStatus', 'FixedOperatingCostPerHorizon','ProductionCost','HandlingCost','OutboundTransportationCost',
                'ProductionCapacityUtilization','ThoughputLevel', 'HandlingCapacityPerHorizon','HandlingCapacityUtilization','StorageCapacity','TurnEstimatedInventory','EndOfPeriodInventory','StorageCapacityUtilization',
                'MaxDistance', 'WeightedAverageDistance']]\
                .to_excel(writer, sheet_name='Output_CostSummaryBySite', index=False)
productline_cost_summary.insert(0, 'Scenario', scenario)
productline_cost_summary.to_excel(writer, sheet_name='Output_CostSummaryByProductLine', index=False)
        
sc_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','ProductType','Period','Quantity','TransportationCost','HandlingCost']].to_excel(writer, sheet_name='Output_CustomerFlows', index=False)
ss_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','Origin','Destination','ProductName','ProductType','Period','Quantity','TransportationCost']].to_excel(writer, sheet_name='Output_IntersiteFlows', index=False)
prod_flow_res_df_1.assign(**{'Scenario': scenario})[['Scenario','FactoryName', 'ProductionLine', 'ProductName','ProductType', 'Period', 'Quantity', 'ProductionCost']].to_excel(writer, sheet_name='Output_ProductionFlows', index=False)
inv_level_res_df_1.assign(**{'Scenario': scenario})[['Scenario','SiteName','ProductName','ProductType','Period','EndOfPeriodInventory','TurnEstimatedInventory']].to_excel(writer, sheet_name='Output_InventoryLevel', index=False)

### need further cleaning format before send to client
### input data tables
periods_df.to_excel(writer, sheet_name='Input_PeriodList', index=False)
products_df.to_excel(writer, sheet_name='Input_ProductList', index=False)
customers_df.to_excel(writer, sheet_name='Input_CustomerList', index=False)
sites_df.to_excel(writer, sheet_name='Input_SiteList', index=False)
productionlines_df.to_excel(writer, sheet_name='Input_ProductLineList', index=False)
trans_policies_df.to_excel(writer, sheet_name='Input_TransportationPolicy', index=False)
prod_policies_df.to_excel(writer, sheet_name='Input_ProductionPolicy', index=False)
inv_df.to_excel(writer, sheet_name='Input_InventoryPolicy', index=False)
demand_df.to_excel(writer, sheet_name='Input_CustomerDemand', index=False)
hist_outbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentFactoryToCust', index=False)
hist_inbound_trans_df.to_excel(writer, sheet_name='Input_HistShipmentAmongFactory', index=False)
hist_prod_df.to_excel(writer, sheet_name='Input_HistProduction', index=False)

writer.save()
writer.close()

c:\Users\52427\.conda\envs\gurobienv\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [3]:
time.time()

### Visualization

In [1]:
### load package
import pandas as pd
pd.set_option('display.max_rows', 500)
import numpy as np
import time
import plotly.graph_objects as go
import plotly.express as px
token = open(".mapbox_token").read() # you will need your own token


model_sanitized_path = 'C:/Users/52427/Documents/Cases/Case11_Baixiang/Baixiang_supply_chain_optimization/model_sanitized/'

file_name = model_sanitized_path + 'model_output_20220620_Unconstrainted+RemoveHandlingCons.xlsx'
xls = pd.ExcelFile(file_name)

### input table
periods_df = pd.read_excel(xls, 'Input_PeriodList')
customers_df = pd.read_excel(xls, 'Input_CustomerList')
sites_df = pd.read_excel(xls, 'Input_SiteList')
productionlines_df = pd.read_excel(xls, 'Input_ProductLineList')
products_df = pd.read_excel(xls, 'Input_ProductList')
demand_df = pd.read_excel(xls, 'Input_CustomerDemand')

### output table
sc_flow_res_df_1 = pd.read_excel(xls, 'Output_CustomerFlows')
ss_flow_res_df_1 = pd.read_excel(xls, 'Output_IntersiteFlows')
prod_flow_res_df_1 = pd.read_excel(xls, 'Output_ProductionFlows')


xls.close()

In [54]:
### plot df preparation
sc_flow_plot_df = sc_flow_res_df_1.groupby(['Origin', 'Destination'])['Quantity'].sum().reset_index()\
                    .merge(customers_df[['CustomerName','Longitude','Latitude']].rename({'CustomerName':'Destination','Longitude':'end_lon','Latitude':'end_lat'}, axis=1), on='Destination')\
                        .merge(sites_df[['SiteName','Longitude','Latitude']].rename({'SiteName':'Origin','Longitude':'start_lon','Latitude':'start_lat'}, axis=1), on='Origin')\
                            .assign(Path = lambda x: x['Origin']+' to '+x['Destination'])

ss_flow_plot_df = ss_flow_res_df_1.groupby(['Origin', 'Destination'])['Quantity'].sum().reset_index()\
                    .merge(sites_df[['SiteName','Longitude','Latitude']].rename({'SiteName':'Destination','Longitude':'end_lon','Latitude':'end_lat'}, axis=1), on='Destination')\
                        .merge(sites_df[['SiteName','Longitude','Latitude']].rename({'SiteName':'Origin','Longitude':'start_lon','Latitude':'start_lat'}, axis=1), on='Origin')\
                            .assign(Path = lambda x: x['Origin']+' to '+x['Destination'])

prod_flow_plot_df = prod_flow_res_df_1.groupby(['FactoryName'])['Quantity'].sum().reset_index()\
                        .merge(sites_df[['SiteName','Longitude','Latitude']].rename({'SiteName':'FactoryName'}, axis=1), on='FactoryName')

demand_plot_df = demand_df.groupby(['CustomerName'])['CustomerDemand'].sum().reset_index()\
                    .merge(customers_df[['CustomerName','Longitude','Latitude']], on='CustomerName')

In [56]:
### plot customers
fig = px.scatter_mapbox(demand_plot_df, lat="Latitude", lon="Longitude", size="CustomerDemand", 
                        hover_name="CustomerName", hover_data=["CustomerName"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
                        
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [57]:
### plot sites
fig = px.scatter_mapbox(prod_flow_plot_df, lat="Latitude", lon="Longitude", size="Quantity", 
                        hover_name="FactoryName", hover_data=["FactoryName"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
                        
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [76]:
### prepare customer flows plot input
lats = []
lons = []
names = []
colors = []
for index, row in sc_flow_plot_df.iterrows():
    lats = np.append(lats, row['start_lat'])
    lats = np.append(lats, row['end_lat'])
    lats = np.append(lats, None)

    lons = np.append(lons, row['start_lon'])
    lons = np.append(lons, row['end_lon'])
    lons = np.append(lons, None)

    colors = np.append(colors, row['Origin'])
    colors = np.append(colors, row['Origin'])
    colors = np.append(colors, row['Origin'])

    names = np.append(names, row['Path'])
    names = np.append(names, row['Path'])
    names = np.append(names, None)

### plot customer flows
fig = px.line_mapbox(lat=lats, lon=lons, color=colors, hover_name=names, zoom=3)
fig.update_traces(line=dict(width=1), opacity=0.8)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [75]:
### prepare intersite flows plot input
lats = []
lons = []
names = []
colors = []
sizes = []
for index, row in ss_flow_plot_df.iterrows():
    lats = np.append(lats, [row['start_lat'],row['end_lat'],None])

    lons = np.append(lons, [row['start_lon'],row['end_lon'],None])

    colors = np.append(colors, [row['Origin']]*3)

    names = np.append(names, [row['Path'], row['Path'], None])

    sizes = np.append(sizes, [row['Quantity'] / ss_flow_plot_df['Quantity'].max()]*3)

### plot customer flows
fig = px.line_mapbox(lat=lats, lon=lons, color=colors, hover_name=names, zoom=3)
fig.update_traces(line=dict(width=1), opacity=0.8)
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [34]:
### plot customer flows - ugly plot - discard
fig = go.Figure()

flight_paths = []
for i in range(len(sc_flow_plot_df)):
    fig.add_trace(
        go.Scattergeo(
            lon = [sc_flow_plot_df['start_lon'][i], sc_flow_plot_df['end_lon'][i]],
            lat = [sc_flow_plot_df['start_lat'][i], sc_flow_plot_df['end_lat'][i]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = 1,
        )
    )

fig.update_layout(
    title_text = 'Customer Flows',
    showlegend = False,
    geo = dict(
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

In [72]:
### plot customer flows - slow plot - discard
fig = go.Figure(go.Scattermapbox(
    mode = "markers",
    lon = customers_df['Longitude'],
    lat = customers_df['Latitude'],
    marker = {'size': 1}))

color_dict = dict(zip(sc_flow_plot_df['Origin'].unique(), px.colors.qualitative.Plotly[0:len(sc_flow_plot_df['Origin'].unique())+1]))

for i in range(len(sc_flow_plot_df)):
    fig.add_trace(
        go.Scattermapbox(
            lon = [sc_flow_plot_df['start_lon'][i], sc_flow_plot_df['end_lon'][i]],
            lat = [sc_flow_plot_df['start_lat'][i], sc_flow_plot_df['end_lat'][i]],
            mode = 'lines',
            line = dict(width = 1, color = color_dict[sc_flow_plot_df['Origin'][i]]),
            opacity = 1,
        )
    )

fig.update_layout(title_text = 'Customer Flows',
                    showlegend = False,
                    mapbox_style="dark", 
                    mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()